In [2]:
import psycopg2
import numpy

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from geoalchemy2 import Geometry, WKTElement
from shapely.geometry import Point

In [3]:
# Define a database name (we're using a dataset on births, so we'll call it birth_db)
# Set your postgres username
dbname = 'map_the_vote'
username = 'codyschank' # change this to your username

engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

postgres://codyschank@localhost/map_the_vote
True


In [4]:
# Connect to make queries using psycopg2
con = None
con = psycopg2.connect(database = dbname, user = username)

In [ ]:
# intersect with districts where I have voter files
# check that both of these geom fields have indexes
sql_query = """
CREATE TABLE select_all_addresses AS
SELECT b.* FROM us_congressional_districts a, addresses_table_tx_no_dupes b 
    WHERE ST_Intersects(a.geom,b.geom) AND a.geoid IN ('4821','4825','4810');
"""
engine.execute(sql_query)

In [4]:
sql_query = """
CREATE INDEX vf_street_address_idx ON voter_file_all (vf_street_address);
"""
engine.execute(sql_query)

In [5]:
sql_query = """
CREATE INDEX oa_street_address_idx ON select_all_addresses (oa_street_address);
"""
engine.execute(sql_query)

In [5]:
# Join open addresses to voter files
sql_query = """
CREATE TABLE voters_join AS
SELECT a.geom, a.oa_lon, a.oa_lat, a.oa_number, a.oa_postcode, a.oa_street_address, a.oa_street, b.* FROM select_all_addresses a LEFT JOIN voter_file_all b ON a.oa_street_address = b.vf_street_address;
"""
engine.execute(sql_query)

In [6]:
# create table of voters not joined to open addresses
sql_query = """
CREATE TABLE addresses_not_joined AS
SELECT geom, oa_lon, oa_lat, oa_number, oa_postcode, oa_street_address, oa_street FROM voters_join WHERE vf_voter_file_vanid IS NULL;
"""
engine.execute(sql_query)

In [7]:
# voter_join is now only the voters and addresses joined correctly
sql_query = """
DELETE FROM voters_join
WHERE vf_voter_file_vanid IS NULL;
"""
engine.execute(sql_query)

In [8]:
sql_query = """
CREATE TABLE voters_not_joined AS
SELECT a.*, b.geom FROM voter_file_all a LEFT JOIN select_all_addresses b ON a.vf_street_address = b.oa_street_address;
"""
engine.execute(sql_query)

In [9]:
# delete voters who were joined, left over is voters not joined
sql_query = """
DELETE FROM voters_not_joined
WHERE geom IS NOT NULL;
"""
engine.execute(sql_query)

In [10]:
sql_query = """
ALTER TABLE voters_not_joined
DROP geom;
"""
engine.execute(sql_query)

In [11]:
sql_query = """
SELECT COUNT(*) FROM voters_not_joined;
"""
pd.read_sql_query(sql_query,con)

,count
0,238708


In [37]:
# STOP TO DO Fuzzy Matching
# In Geocoding and RDI Check, now use the tables after fuzzy matching
# STOP TO DO GEOCODING OF VOTERS NOT JOINED AND RDI CHECK OF ADDRESSES NOT JOINED

In [5]:
# create table voters_not_joined_geocoded by joining geocoded voters not joined with original table that identified them, tx25_join_check2
sql_query = """
CREATE TABLE voters_not_joined_geocoded AS
SELECT a.*, b.geom FROM voters_still_not_joined a LEFT JOIN smarty_streets_geocode b ON a.vf_street_address = b.vf_street_address;
"""
engine.execute(sql_query)

In [6]:
sql_query = """
SELECT COUNT(*) FROM voters_not_joined_geocoded WHERE geom IS NULL;
"""
pd.read_sql_query(sql_query,con)

,count
0,11710


In [7]:
# now append voters_not_joined_geocoded to voters_join
# I should reconsider doing an insert here, since it means I will need to redo the original join if I want to change anything
# maybe I can add a field that tells how the record was geocoded, althought, that should be recorded by whether the row has an oa_address
sql_query = """
INSERT INTO voters_join (geom, vf_voter_file_vanid, vf_sex, vf_age, vf_street_address, vf_multi_unit, vf_cntyvtd)
SELECT geom, vf_voter_file_vanid, vf_sex, vf_age, vf_street_address, vf_multi_unit, vf_cntyvtd
FROM voters_not_joined_geocoded;
"""
engine.execute(sql_query)

In [8]:
# now append oa_fuzzy_join to voters_join
# I should reconsider doing an insert here, since it means I will need to redo the original join if I want to change anything
# maybe I can add a field that tells how the record was geocoded, althought, that should be recorded by whether the row has an oa_address
sql_query = """
INSERT INTO voters_join (geom, vf_voter_file_vanid, vf_sex, vf_age, vf_street_address, vf_multi_unit, vf_cntyvtd)
SELECT geom, vf_voter_file_vanid, vf_sex, vf_age, vf_street_address, vf_multi_unit, vf_cntyvtd
FROM oa_fuzzy_join;
"""
engine.execute(sql_query)

In [9]:
sql_query = """
SELECT COUNT(*) FROM voters_join;
"""
pd.read_sql_query(sql_query,con)

,count
0,1441170


In [10]:
sql_query = """
CREATE TABLE voters_not_joined_geocoded_3081 AS
    SELECT * FROM voters_not_joined_geocoded;
"""
engine.execute(sql_query)

In [11]:
sql_query = """
ALTER TABLE voters_not_joined_geocoded_3081 
   ALTER COLUMN geom 
   TYPE Geometry(Point, 3081) 
   USING ST_Transform(geom, 3081);
"""
engine.execute(sql_query)

In [12]:
sql_query = """
CREATE INDEX voters_not_joined_geocoded_3081_gix ON voters_not_joined_geocoded_3081 USING GIST (geom);
"""
engine.execute(sql_query)

In [13]:
# Copy and project tables that I need for DWithin, calculate indices
# I could have used this projection from the beginning
sql_query = """
CREATE TABLE addresses_still_not_joined_3081 AS
    SELECT * FROM addresses_still_not_joined;
"""
engine.execute(sql_query)

In [14]:
sql_query = """
ALTER TABLE addresses_still_not_joined_3081 
   ALTER COLUMN geom 
   TYPE Geometry(Point, 3081) 
   USING ST_Transform(geom, 3081);
"""
engine.execute(sql_query)

In [15]:
sql_query = """
CREATE INDEX addresses_still_not_joined_3081_gix ON addresses_still_not_joined_3081 USING GIST (geom);
"""
engine.execute(sql_query)

In [16]:
# this table might come in handy, but right now don't have a need for it
sql_query = """
CREATE TABLE voters_join_3081 AS
    SELECT * FROM voters_join;
"""
engine.execute(sql_query)

In [17]:
sql_query = """
ALTER TABLE voters_join_3081 
   ALTER COLUMN geom 
   TYPE Geometry(Point, 3081) 
   USING ST_Transform(geom, 3081);
"""
engine.execute(sql_query)

In [18]:
sql_query = """
CREATE INDEX voters_join_3081_gix ON voters_join_3081 USING GIST (geom);
"""
engine.execute(sql_query)

In [19]:
sql_query = """
ALTER TABLE addresses_still_not_joined_3081
ADD COLUMN join_mask integer;
"""
engine.execute(sql_query)

In [20]:
sql_query = """
UPDATE addresses_still_not_joined_3081
SET "join_mask" = 0;
"""
engine.execute(sql_query)

In [21]:
sql_query = """
UPDATE addresses_still_not_joined_3081 dst
SET "join_mask" = 1
FROM voters_not_joined_geocoded_3081 src
WHERE ST_DWITHIN(src.geom,dst.geom,10);
"""
engine.execute(sql_query)

In [22]:
sql_query = """
SELECT COUNT(*) FROM addresses_still_not_joined_3081 WHERE join_mask = 1;
"""
pd.read_sql_query(sql_query,con)

,count
0,1131


In [5]:
## START HERE, drop tables I am replacing first
## Drop smarty_streets_rdi_check?
sql_query = """
CREATE TABLE final_addresses_not_joined AS 
SELECT a.*, b.residential, b.vacant, b.active FROM addresses_still_not_joined_3081 a LEFT JOIN smarty_streets_rdi_check_new b ON a.oa_street_address = b.oa_street_address;
"""
engine.execute(sql_query)

In [8]:
sql_query = """
DELETE FROM final_addresses_not_joined
WHERE residential != 'Residential' OR join_mask = 1 OR active = 'N' OR vacant = 'Y';
"""
engine.execute(sql_query)

In [9]:
sql_query = """
SELECT COUNT(*) FROM final_addresses_not_joined;
"""
pd.read_sql_query(sql_query,con)

,count
0,186462


In [10]:
sql_query = """
CREATE INDEX final_addresses_not_joined_gix ON final_addresses_not_joined USING GIST (geom);
"""
engine.execute(sql_query)

In [ ]:
# project vtds_tx to 3081

In [27]:
sql_query = """
CREATE TABLE vtds_tx_3081 AS
    SELECT * FROM vtds_tx;
"""
engine.execute(sql_query)

In [29]:
sql_query = """
ALTER TABLE vtds_tx_3081 
   ALTER COLUMN geom 
   TYPE Geometry(Polygon, 3081) 
   USING ST_Transform(geom, 3081);
"""
engine.execute(sql_query)

In [30]:
sql_query = """
CREATE INDEX vtds_tx_3081_gix ON vtds_tx_3081 USING GIST (geom);
"""
engine.execute(sql_query)

In [12]:
# spatial join the cntyvtd field to final_addresses_not_joined
sql_query = """
CREATE TABLE final_addresses_not_joined_vtd AS
SELECT p1.*,p2.cntyvtd FROM final_addresses_not_joined p1, vtds_tx_3081 p2 WHERE ST_WITHIN(p1.geom, p2.geom)
"""
engine.execute(sql_query)

In [13]:
sql_query = """
SELECT * FROM final_addresses_not_joined_vtd LIMIT 5;
"""
pd.read_sql_query(sql_query,con)

,geom,oa_lon,oa_lat,oa_number,oa_postcode,oa_street_address,oa_street,join_mask,residential,vacant,active,cntyvtd
0,0101000020090C000054D3106E3A933141382E972997FD...,-98.412000,30.704547,104,78611,104 CR 119A 78611,CR 119A,0,Residential,N,Y,530008
1,0101000020090C0000331422971D973141580AAE3DEC07...,-98.401391,30.716370,1700,78611,1700 CR 118 78611,CR 118,0,Residential,N,Y,530008
2,0101000020090C00003573906C069631418C18F51CE3F1...,-98.404737,30.690915,1814,78611,1814 CR 134 78611,CR 134,0,Residential,N,Y,530008
3,0101000020090C00006052BC4FA997314187836DFEA708...,-98.399915,30.717201,113,78611,113 REGINAS TRL 78611,REGINAS TRL,0,Residential,N,Y,530008
4,0101000020090C00007B6775BD7096314167A0E27ED805...,-98.403239,30.713990,1013,78611,1013 APACHE DR 78611,APACHE DR,0,Residential,N,Y,530008
